<a href="https://colab.research.google.com/github/jedrexd/WTUM_11_2022/blob/main/Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
!git clone https://github.com/jedrexd/WTUM_11_2022

Cloning into 'WTUM_11_2022'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 68 (delta 36), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (68/68), done.


In [57]:
df = pd.read_csv("/content/WTUM_11_2022/train.csv", header=0)
df = df.iloc[:,1:]
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [58]:
df.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [59]:
df.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [60]:
# MSSubClass: Identifies the type of dwelling involved in the sale. - kategorie
df['MSSubClass'] = LabelEncoder().fit_transform(df['MSSubClass'].astype('str'))

# MSZoning: Identifies the general zoning classification of the sale. - kategorie
df['MSZoning'] = LabelEncoder().fit_transform(df['MSZoning'].astype('str'))

# LotFrontage: Linear feet of street connected to property - zmienna liczbowa, skalujemy według rozkładu normalnego
# NA oznacza 0
df['LotFrontage'] = df['LotFrontage'].replace({"NA": 0})
df['LotFrontage'] = StandardScaler().fit_transform(np.array(df['LotFrontage']).reshape(-1, 1));

# LotArea: Lot size in square feet - zmienna liczbowa, skalujemy według rozkładu normalnego
df['LotArea'] = StandardScaler().fit_transform(df['LotArea'].values.reshape(-1, 1))

# Street: Type of road access to property - kategoria
df['Street'] = LabelEncoder().fit_transform(df['Street'].astype('str'))

# Alley: Type of alley access to property - kategoria
df['Alley'] = LabelEncoder().fit_transform(df['Alley'].astype('str'))

# LotShape: General shape of property - kategoria, zmienna jakościowa
lotShapeDict = {"Reg": 1, "IR1": 2, "IR2": 3, "IR3": 4}
df['LotShape'] = df['LotShape'].replace(lotShapeDict)

# LandContour: Flatness of the property - kategoria
df['LandContour'] = LabelEncoder().fit_transform(df['LandContour'].astype('str'))

# LotConfig: Lot configuration - kategoria
df['LotConfig'] = LabelEncoder().fit_transform(df['LotConfig'].astype('str'))

# LandSlope: Slope of property - kategoria
df['LandSlope'] = LabelEncoder().fit_transform(df['LandContour'].astype('str'))

# Neighborhood: Physical locations within Ames city limits - kategoria
df['Neighborhood'] = LabelEncoder().fit_transform(df['Neighborhood'].astype('str'))

# Condition1: Proximity to various conditions
# Condition2: Proximity to various conditions (if more than one is present) - kategorie
conditionDict = {"Artery": 1, "Feedr": 2, "Norm": 3,	"RRNn": 4, "RRAn": 5, "PosN": 6, "PosA": 7, "RRNe": 8, "RRAe": 9}
df['Condition1'] = df['Condition1'].replace(conditionDict)
df['Condition2'] = df['Condition2'].replace(conditionDict)

# Utilities: Type of utilities available - kategoria, zmienna jakościowa - każda kolejna kategoria oznacza wyższy standard
utilitiesDict = {"AllPub": 3, "NoSewr": 2, "NoSeWa": 1, "ELO": 0}
df['Utilities'] = df['Utilities'].replace(utilitiesDict);

# BldgType: Type of dwelling - kategoria
df['BldgType'] = LabelEncoder().fit_transform(df['BldgType'].astype('str'))

# HouseStyle: Style of dwelling - kategoria
df['HouseStyle'] = LabelEncoder().fit_transform(df['HouseStyle'].astype('str'))

# YearBuilt, YearRemodAdd, OverallCond, OverallQual: Zmienne liczbowe - uznaliśmy, że nie ma potrzeby ich przekształcać

In [61]:
print(df['GarageCars'].isna().sum())

0


In [62]:
print(df['GarageArea'].isna().sum())
df['GarageArea'] = StandardScaler().fit_transform(df['GarageArea'].values.reshape(-1,1))

0


In [63]:
print(df['GarageQual'].isna().sum())
unq_vals = df['GarageQual'].unique().astype('str')
df['GarageQual'] = df['GarageQual'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, np.nan: 0})

81


In [64]:
print(df['GarageCond'].isna().sum())
print(df['GarageCond'].unique())
df['GarageCond'] = df['GarageCond'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, np.nan: 0})

81
['TA' 'Fa' nan 'Gd' 'Po' 'Ex']


In [65]:
print(df['PavedDrive'].isna().sum())
df['PavedDrive'] = LabelEncoder().fit_transform(df['PavedDrive'])

0


In [66]:
print(df['WoodDeckSF'].isna().sum())
df['WoodDeckSF'] = StandardScaler().fit_transform(df['WoodDeckSF'].values.reshape(-1,1))

0


In [67]:
print(df['OpenPorchSF'].isna().sum())
df['OpenPorchSF'] = StandardScaler().fit_transform(df['OpenPorchSF'].values.reshape(-1,1))

0


In [68]:
print(df['EnclosedPorch'].isna().sum())
df['EnclosedPorch'] = StandardScaler().fit_transform(df['EnclosedPorch'].values.reshape(-1,1))

0


In [69]:
print(df['3SsnPorch'].isna().sum())
df['3SsnPorch'] = StandardScaler().fit_transform(df['3SsnPorch'].values.reshape(-1,1))

0


In [70]:
print(df['ScreenPorch'].isna().sum())
df['ScreenPorch'] = StandardScaler().fit_transform(df['ScreenPorch'].values.reshape(-1,1))

0


In [71]:
print(df['PoolArea'].isna().sum())
df['PoolArea'] = StandardScaler().fit_transform(df['PoolArea'].values.reshape(-1,1))

0


In [72]:
print(df['PoolQC'].isna().sum())
print(df['PoolQC'].unique())
df['PoolQC'] = df['PoolQC'].replace({'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, np.nan: 0})

1453
[nan 'Ex' 'Fa' 'Gd']


In [73]:
print(df['Fence'].isna().sum())
print(df['Fence'].unique())
df['Fence'] = df['Fence'].replace({'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, np.nan: 0})

1179
[nan 'MnPrv' 'GdWo' 'GdPrv' 'MnWw']


In [105]:
print(df['MiscFeature'].isna().sum())
print(df['MiscFeature'].unique())
df['MiscFeature'] = df['MiscFeature'].replace({'Elev': 5, 'Gar2': 4, 'Othr': 3, 'Shed': 2, 'TenC': 1, np.nan: 0})

1406
[nan 'Shed' 'Gar2' 'Othr' 'TenC']


In [75]:
print(df['MiscVal'].isna().sum())
df['MiscVal'] = StandardScaler().fit_transform(df['MiscVal'].values.reshape(-1,1))

0


In [76]:
print(df['MoSold'].isna().sum())

0


In [77]:
print(df['YrSold'].isna().sum())

0


In [78]:
print(df['SaleType'].isna().sum())
df['SaleType'] = LabelEncoder().fit_transform(df['SaleType'])

0


In [79]:
print(df['SaleCondition'].isna().sum())
df['SaleCondition'] = LabelEncoder().fit_transform(df['SaleCondition'])

0


In [80]:
print(df['SalePrice'].isna().sum())
df['MiscVal'] = np.log(df['MiscVal'])

0


/usr/local/lib/python3.7/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [81]:
# kolumny 21 - 40

df20_40 = df.iloc[:,20:40]  # będę pracował na kopii, potem ją wrzucę do głównego df

In [82]:
# 21 kolumna - RoofStyle

col = df20_40['RoofStyle']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

np_col = np.array(col)
print(f'Nazwy unikatowych wartości: {np.unique(np_col)}')

# zamiana string na int, zaczynajac od 0 i idąc alfabetycznie
new_names = {'Flat': 0,
             'Gable': 1,
             'Gambrel': 2,
             'Hip': 3,
             'Mansard': 4,
             'Shed': 5}

col_num = col.replace(new_names)
print(f'Sposób zamiany kategorii {new_names}')

df20_40['RoofStyle'] = col_num

Czy są NaNy: False
Nazwy unikatowych wartości: ['Flat' 'Gable' 'Gambrel' 'Hip' 'Mansard' 'Shed']
Sposób zamiany kategorii {'Flat': 0, 'Gable': 1, 'Gambrel': 2, 'Hip': 3, 'Mansard': 4, 'Shed': 5}


In [83]:
# 22 kolumna - RoofMatl

col = df20_40['RoofMatl']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

np_col = np.array(col)
print(f'Nazwy unikatowych wartości: {np.unique(np_col)}')

# zamiana string na int, zaczynajac od 0 i idąc alfabetycznie
new_names = {'ClyTile': 0,
             'CompShg': 1,
             'Membran': 2,
             'Metal': 3,
             'Roll': 4,
             'Tar&Grv': 5,
             'WdShake': 6,
             'WdShngl': 7}

col_num = col.replace(new_names)
print(f'Sposób zamiany kategorii {new_names}')

df20_40['RoofMatl'] = col_num

Czy są NaNy: False
Nazwy unikatowych wartości: ['ClyTile' 'CompShg' 'Membran' 'Metal' 'Roll' 'Tar&Grv' 'WdShake'
 'WdShngl']
Sposób zamiany kategorii {'ClyTile': 0, 'CompShg': 1, 'Membran': 2, 'Metal': 3, 'Roll': 4, 'Tar&Grv': 5, 'WdShake': 6, 'WdShngl': 7}


In [84]:
# 23 kolumna - Exterior1st


col = df20_40['Exterior1st']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

np_col = np.array(col)
print(f'Nazwy unikatowych wartości: {np.unique(np_col)}')

# zamiana string na int, robię to LabelEncoderem ponieważ nie jest to zmienna jakościowa (cech nie da się uporządkować)
col_num = LabelEncoder().fit_transform(col.astype('str'))

df20_40['Exterior1st'] = col_num

Czy są NaNy: False
Nazwy unikatowych wartości: ['AsbShng' 'AsphShn' 'BrkComm' 'BrkFace' 'CBlock' 'CemntBd' 'HdBoard'
 'ImStucc' 'MetalSd' 'Plywood' 'Stone' 'Stucco' 'VinylSd' 'Wd Sdng'
 'WdShing']


In [85]:
# 24 kolumna - Exterior2nd

col = df20_40['Exterior2nd']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

np_col = np.array(col)
print(f'Nazwy unikatowych wartości: {np.unique(np_col)}')

# zamiana string na int, robię to LabelEncoderem ponieważ nie jest to zmienna jakościowa (cech nie da się uporządkować)
col_num = LabelEncoder().fit_transform(col.astype('str'))

df20_40['Exterior2nd'] = col_num

Czy są NaNy: False
Nazwy unikatowych wartości: ['AsbShng' 'AsphShn' 'Brk Cmn' 'BrkFace' 'CBlock' 'CmentBd' 'HdBoard'
 'ImStucc' 'MetalSd' 'Other' 'Plywood' 'Stone' 'Stucco' 'VinylSd'
 'Wd Sdng' 'Wd Shng']


In [86]:
# 25 kolumna - MasVnrType: Masonry veneer type

col = df20_40['MasVnrType']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy
if col.isnull().values.any():
  col = col.fillna('-1')

np_col = np.array(col)
print(f'Nazwy unikatowych wartości: {np.unique(np_col)}')

# zamiana string na int, cech nie da się uporządkować ale robię Series.replace() ze względu na to, że chcę aby NaNy miały wartość -1
col_names = {'-1': -1,
             'BrkCmn': 0,
             'BrkFace': 1,
             'CBlock': 2,
             'None': 3,
             'Stone': 4}

col_num = col.replace(col_names)

df20_40['MasVnrType'] = col_num

Czy są NaNy: True
Nazwy unikatowych wartości: ['-1' 'BrkCmn' 'BrkFace' 'None' 'Stone']


In [87]:
# 26 kolumna - MasVnrArea: Masonry veneer area in square feet

col = df20_40['MasVnrArea']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy
if col.isnull().values.any():
  col = col.fillna(col.mean())

# kolumna posiada ciągłe wartości liczbowe, więc przeskalowuje funkcją StandardScaler() z domyślnymi parametrami (przesunięcie o średnią próbki i podzielenie przez odchylenie standardowe)
np_col = col.values
np_col = StandardScaler().fit_transform(np_col.reshape([-1,1]))

df20_40['MasVnrArea'] = np_col

Czy są NaNy: True


In [88]:
# 27 kolumna - ExterQual

col = df20_40['ExterQual']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

# zmienna jakościowa, zamiana string na int w ten sposób, że wieksza liczba == lepiej
new_names = {'Po': 0,
             'Fa': 1,
             'TA': 2,
             'Gd': 3,
             'Ex': 4}

col_num = col.replace(new_names)

df20_40['ExterQual'] = col_num

Czy są NaNy: False


In [89]:
# 28 kolumna - ExterCond

col = df20_40['ExterCond']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

# zmienna jakościowa, zamiana string na int w ten sposób, że wieksza liczba == lepiej
new_names = {'Po': 0,
             'Fa': 1,
             'TA': 2,
             'Gd': 3,
             'Ex': 4}

col_num = col.replace(new_names)

df20_40['ExterCond'] = col_num

Czy są NaNy: False


In [90]:
# 29 kolumna - Foundation

col = df20_40['Foundation']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

# zamiana string na int, robię to LabelEncoderem ponieważ nie jest to zmienna jakościowa (cech nie da się uporządkować)
col_num = LabelEncoder().fit_transform(col.astype('str'))


df20_40['Foundation'] = col_num

Czy są NaNy: False


In [91]:
# 30 kolumna - BsmtQual: Evaluates the height of the basement
# zakladam ze null to brak piwnicy

col = df20_40['BsmtQual']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy
col = col.fillna('NA')

# zmienna jakościowa, zamiana string na int w ten sposób, że wieksza liczba == lepiej
new_names = {'NA': 0,
             'Po': 1,
             'Fa': 2,
             'TA': 3,
             'Gd': 4,
             'Ex': 5}

col_num = col.replace(new_names)

df20_40['BsmtQual'] = col_num

Czy są NaNy: True


In [92]:
# 31 kolumna - BsmtCond: Evaluates the general condition of the basement
# zakladam ze null to brak piwnicy

col = df20_40['BsmtCond']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy
col = col.fillna('NA')

# zmienna jakościowa, zamiana string na int w ten sposób, że wieksza liczba == lepiej
new_names = {'NA': 0,
             'Po': 1,
             'Fa': 2,
             'TA': 3,
             'Gd': 4,
             'Ex': 5}

col_num = col.replace(new_names)

df20_40['BsmtCond'] = col_num

Czy są NaNy: True


In [93]:
# 32 kolumna - BsmtExposure: Refers to walkout or garden level walls
# zakladam ze null to brak piwnicy

col = df20_40['BsmtExposure']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy
col = col.fillna('NA')

# zmienna jakościowa, zamiana string na int w ten sposób, że wieksza liczba == lepiej
new_names = {'NA': 0,
             'No': 1,
             'Mn': 2,
             'Av': 3,
             'Gd': 4}

col_num = col.replace(new_names)

df20_40['BsmtExposure'] = col_num

Czy są NaNy: True


In [94]:
# 33 kolumna - BsmtFinType1: Rating of basement finished area

col = df20_40['BsmtFinType1']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy
col = col.fillna('NA')

# zmienna jakościowa, zamiana string na int w ten sposób, że wieksza liczba == lepiej
new_names = {'NA': 0,
             'Unf': 1,
             'LwQ': 2,
             'Rec': 3,
             'BLQ': 4,
             'ALQ': 5,
             'GLQ': 6}

col_num = col.replace(new_names)

df20_40['BsmtFinType1'] = col_num

Czy są NaNy: True


In [95]:
# 34 kolumna - BsmtFinSF1: Type 1 finished square feet

col = df20_40['BsmtFinSF1']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

# kolumna posiada ciągłe wartości liczbowe, więc przeskalowuje funkcją StandardScaler() z domyślnymi parametrami (przesunięcie o średnią próbki i podzielenie przez odchylenie standardowe)
np_col = col.values
np_col = StandardScaler().fit_transform(np_col.reshape([-1,1]))

df20_40['BsmtFinSF1'] = np_col

Czy są NaNy: False


In [96]:
# 35 kolumna - BsmtFinType2: Rating of basement finished area (if multiple types)


col = df20_40['BsmtFinType2']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy
col = col.fillna('NA')

# zmienna jakościowa, zamiana string na int w ten sposób, że wieksza liczba == lepiej
new_names = {'NA': 0,
             'Unf': 1,
             'LwQ': 2,
             'Rec': 3,
             'BLQ': 4,
             'ALQ': 5,
             'GLQ': 6}

col_num = col.replace(new_names)

df20_40['BsmtFinType2'] = col_num

Czy są NaNy: True


In [97]:
# 36 kolumna - BsmtFinSF2: Type 2 finished square feet

col = df20_40['BsmtFinSF2']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

# kolumna posiada ciągłe wartości liczbowe, więc przeskalowuje funkcją StandardScaler() z domyślnymi parametrami (przesunięcie o średnią próbki i podzielenie przez odchylenie standardowe)
np_col = col.values
np_col = StandardScaler().fit_transform(np_col.reshape([-1,1]))

df20_40['BsmtFinSF2'] = np_col

Czy są NaNy: False


In [98]:
# 37 kolumna - BsmtUnfSF: Unfinished square feet of basement area

col = df20_40['BsmtUnfSF']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

# kolumna posiada ciągłe wartości liczbowe, więc przeskalowuje funkcją StandardScaler() z domyślnymi parametrami (przesunięcie o średnią próbki i podzielenie przez odchylenie standardowe)
np_col = col.values
np_col = StandardScaler().fit_transform(np_col.reshape([-1,1]))

df20_40['BsmtUnfSF'] = np_col

Czy są NaNy: False


In [99]:
# 38 kolumna - TotalBsmtSF: Total square feet of basement area

col = df20_40['TotalBsmtSF']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

# kolumna posiada ciągłe wartości liczbowe, więc przeskalowuje funkcją StandardScaler() z domyślnymi parametrami (przesunięcie o średnią próbki i podzielenie przez odchylenie standardowe)
np_col = col.values
np_col = StandardScaler().fit_transform(np_col.reshape([-1,1]))

df20_40['TotalBsmtSF'] = np_col

Czy są NaNy: False


In [100]:
# 39 kolumna - Heating: Type of heating

col = df20_40['Heating']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

# zmienna jakościowa, zamiana string na int w ten sposób, że wieksza liczba == lepiej
new_names = {'Wall': 0,
             'Floor': 1,
             'GasA': 2,
             'GasW': 3,
             'Grav': 4,
             'OthW': 5,
             'Wall': 6}

col_num = col.replace(new_names)

df20_40['Heating'] = col_num

Czy są NaNy: False


In [101]:
# 40 kolumna - HeatingQC: Heating quality and condition

col = df20_40['HeatingQC']
print(f'Czy są NaNy: {col.isnull().values.any()}')  # sprawdzam czy są gdzieś w kolumnie NaNy

# zmienna jakościowa, zamiana string na int w ten sposób, że wieksza liczba == lepiej
new_names = {'Po': 0,
             'Fa': 1,
             'TA': 2,
             'Gd': 3,
             'Ex': 4}

col_num = col.replace(new_names)

df20_40['HeatingQC'] = col_num

Czy są NaNy: False


In [102]:
df.iloc[:,20:40] = df20_40

In [103]:
# kolumny 41-60
df2 = df.iloc[:,40:59]

# BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, TotRmsAbvGrd, Fireplaces - kolumny, z którymi nic nie robiłam, bo mają po kilka dyskretnych wartosci, wiec beda dzialac :)
#GarageYrBlt - lata (1900 - 2010), trzeba się zastanowić jeszcze, co zrobić z tą kolumną


#CentralAir - zmienna kategoryczna, dwie wartości Yes, No przerabiamy na 0,1
df2['CentralAir'] = LabelEncoder().fit_transform(df['CentralAir'].astype('str'))

#Electrical - zmienna kategoryczna (5 kategorii) -> zmienna dyskretna (0-4)
df2['Electrical'] = LabelEncoder().fit_transform(df['Electrical'].astype('str'))

#1stFlrSF - powierzchnia pierwszego pietra -> standaryzacja
c3 = df["1stFlrSF"].values 
df2["1stFlrSF"] = StandardScaler().fit_transform(c3.reshape(-1,1)) 

#2ndFlrSF - powierzchnia drugiego pietra -> standaryzacja
c4 = df["2ndFlrSF"].values     
df2["2ndFlrSF"] = StandardScaler().fit_transform(c4.reshape(-1,1)) 

#LowQualFinSF - powierzchnia zlej jakosci -> standaryzacja
c5 = df["LowQualFinSF"].values      
df2["LowQualFinSF"] = StandardScaler().fit_transform(c5.reshape(-1,1)) 

#GrLivAre - powierzchnia jakosci ponad norme -> standaryzacja
c6 = df["GrLivArea"].values   
df2["GrLivArea"] = StandardScaler().fit_transform(c6.reshape(-1,1)) 

#KitchenQual - zmienna kategoryczna (5 kategorii) -> zmienna dyskretna (0-4)
c11 = df["KitchenQual"]
new_names = {
    'Po' : 0,
    'Fa' : 1,
    'TA' : 2,
    'Gd' : 3,
    'Ex' : 4
}
df2["KitchenQual"] = c11.replace(new_names)

#Functional - zmienna jakościowa (8 kategorii w porządku od najgorszego (0) do najlepszego (7)) -> zmienna dyskretna (0-7)
c13 = df["Functional"]
new_names1 = {
    'Sal' : 0,
    'Sev' : 1,
    'Maj2' : 2,
    'Maj1' : 3,
    'Mod' : 4,
    'Min2' : 5,
    'Min1' : 6,
    'Typ' : 7
}
df2["Functional"] = c13.replace(new_names1)

#FireplaceQu - NA - z opisu danych oznacza brak kominka, nadajemy temu kategorię 0
#            - zmienna jakościowa (6 kategorii od najgorszej, czyli braku kominka(0) do najlepszej (5)) -> zmienna dyskretna (0-5)
c15 = df["FireplaceQu"]
c15 = c15.fillna(0)
new_names2 = {
    'NA' : 0,
    'NaN' : 0,
    'Po' : 1,
    'Fa' : 2,
    'TA' : 3,
    'Gd' : 4,
    'Ex' : 5
}
df2["FireplaceQu"] = c15.replace(new_names2)

#GarageType - z opisu danych brak oznacza brak garażu -> nadajemy kategorię 0
#           - zmienna kategoryczna  (7 kategorii) -> zmienna dyskretna (0-6)
c16 = df["GarageType"]
new_names3 = {
    'NA' : 0,
    'NaN' : 0,
    '' : 0,
    'Detchd' : 1,
    'CarPort' : 2,
    'BuiltIn' : 3,
    'Basment' : 4,
    'Attchd' : 5,
    '2Types' : 6
}
df2["GarageType"] = c16.replace(new_names3)

#GarageFinish - z opisu danych NA oznacza brak garaż€ -> nadajemy kategorię 0
#             - zmienna jakościowa (4 kategorie, od najgorszej (0), do najlepszej (3)) -> zmienna dyskretna (0-3)
c18 = df["GarageFinish"]
new_names4 = {
    'NA' : 0,
    'Unf' : 1,
    'RFn' : 2,
    'Fin' : 3
}
df2["GarageFinish"] = c18.replace(new_names4)



In [106]:
df.iloc[:, 40:59] = df2

df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,9,3,-0.208034,-0.207142,1,2,1,3,3,4,...,-0.068692,0,0,0,NaN,2,2008,8,4,208500
1,4,3,0.409895,-0.091886,1,2,1,3,3,2,...,-0.068692,0,0,0,NaN,5,2007,8,4,181500
2,9,3,-0.084449,0.073480,1,2,2,3,3,4,...,-0.068692,0,0,0,NaN,9,2008,8,4,223500
3,10,3,-0.414011,-0.096897,1,2,2,3,3,0,...,-0.068692,0,0,0,NaN,2,2006,8,0,140000
4,9,3,0.574676,0.375148,1,2,2,3,3,2,...,-0.068692,0,0,0,NaN,12,2008,8,4,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,9,3,-0.331620,-0.260560,1,2,1,3,3,4,...,-0.068692,0,0,0,NaN,8,2007,8,4,175000
1456,4,3,0.615871,0.266407,1,2,1,3,3,4,...,-0.068692,0,3,0,NaN,2,2010,8,4,210000
1457,10,3,-0.166839,-0.147810,1,2,1,3,3,4,...,-0.068692,0,4,2,1.600016,5,2010,8,4,266500
1458,4,3,-0.084449,-0.080160,1,2,1,3,3,4,...,-0.068692,0,0,0,NaN,4,2010,8,4,142125
